In [ ]:
from analyst_9000.backend.core.setup import *
from google.cloud import bigquery


def simple_test_query():
    # Αν η env var GOOGLE_APPLICATION_CREDENTIALS είναι σωστά ρυθμισμένη
    client = bigquery.Client()  # χρησιμοποιεί credentials αυτόματα

    QUERY = """
    SELECT
      COUNT(*) as total_orders
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    LIMIT 10
    """
    query_job = client.query(QUERY)  # ξεκινά το query
    result = query_job.result()       # περιμένει να ολοκληρωθεί

    for row in result:
        print("Total orders (check):", row["total_orders"])
if __name__ == "__main__":
    simple_test_query()


In [ ]:
from analyst_9000.backend.core.setup import *
from google.cloud import bigquery
import warnings
warnings.filterwarnings(
    "ignore",
    message="BigQuery Storage module not found, fetch data with the REST endpoint instead."
)
def preview_tables():
    client = bigquery.Client()
    dataset = "bigquery-public-data.thelook_ecommerce"

    tables = ["orders", "order_items", "products", "users"]

    for table in tables:
        print(f"\n===== {table.upper()} =====")
        df = client.query(f"SELECT * FROM `{dataset}.{table}` LIMIT 5").to_dataframe()
        print(df.head())

if __name__ == "__main__":
    preview_tables()


In [ ]:
from analyst_9000.backend.core.setup import *
from google.cloud import bigquery
client = bigquery.Client()
table = client.get_table("bigquery-public-data.thelook_ecommerce.orders")

print("Schema for table:", table.table_id)
for field in table.schema:
    print(f"- {field.name}: {field.field_type}")




In [ ]:
client = bigquery.Client()
df = client.query("SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders`").to_dataframe()
print(df.head(10))
print(df.dtypes)

In [ ]:
df.head()

In [ ]:
from google.cloud import bigquery
from analyst_9000.backend.core.setup import *

from google.cloud import bigquery

def get_table_ddl(table_fqn: str) -> str:
    """
    Returns the DDL of a BigQuery table.
    - If the table is in your project: use INFORMATION_SCHEMA (real DDL)
    - If it's a public dataset: auto-generate DDL from the schema
    """
    client = bigquery.Client()

    # Parse table_fqn → project, dataset, table
    try:
        project, dataset, table = table_fqn.split(".")
    except ValueError:
        raise ValueError("Table FQN must be 'project.dataset.table'")

    # 1️⃣ Try INFORMATION_SCHEMA (works only for tables in your project)
    try:
        query = f"""
        SELECT ddl
        FROM `{project}.{dataset}.INFORMATION_SCHEMA.TABLES`
        WHERE table_name = '{table}'
        """
        rows = list(client.query(query).result())
        if rows and rows[0].get("ddl"):
            return rows[0]["ddl"]
    except Exception:
        pass  # Fall back to manual DDL generation

    # 2️⃣ Fallback for public datasets — create DDL from schema
    bq_table = client.get_table(f"{project}.{dataset}.{table}")

    ddl_lines = []
    ddl_lines.append(f"CREATE TABLE `{project}.{dataset}.{table}` (")
    fields = []

    for field in bq_table.schema:
        field_def = f"  `{field.name}` {field.field_type}"
        if field.mode == "REPEATED":
            field_def += " REPEATED"
        elif field.mode == "REQUIRED":
            field_def += " NOT NULL"
        fields.append(field_def)

    ddl_lines.append(",\n".join(fields))
    ddl_lines.append(");")

    return "\n".join(ddl_lines)


In [ ]:
ddl = get_table_ddl("bigquery-public-data.thelook_ecommerce.orders")
print(ddl)



In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

project_id = "bigquery-public-data"
dataset_id = "thelook_ecommerce"      # π.χ. "thelook_ecommerce"

# 1) List all tables / views under the dataset
sql_tables = f"""
SELECT
  table_catalog,
  table_schema,
  table_name,
  table_type,
  creation_time,
  ddl
FROM
  `{project_id}.{dataset_id}.INFORMATION_SCHEMA.TABLES`
ORDER BY
  table_name;
"""

print("Tables / views in dataset:")
for row in client.query(sql_tables).result():
    print("-", row.table_name, "(type:", row.table_type, ") created:", row.creation_time)

# 2) For each table, list its columns and metadata
sql_columns = f"""
SELECT
  table_name,
  column_name,
  data_type,
  is_nullable,
  ordinal_position,
  is_partitioning_column,
  is_hidden,
  is_system_defined
FROM
  `{project_id}.{dataset_id}.INFORMATION_SCHEMA.COLUMNS`
ORDER BY
  table_name, ordinal_position;
"""

print("\nColumns metadata:")
for row in client.query(sql_columns).result():
    print(f"- Table: {row.table_name}, Column: {row.column_name}, Type: {row.data_type}, Nullable: {row.is_nullable}")


In [1]:
from analyst_9000.backend.core.setup import *
from analyst_9000.backend.core.config import get_settings
import pprint   


settings = get_settings()
settings.bigquery_handler.tables_description

{"timestamp": "2025-12-01 19:41:08,447", "level": "INFO", "logger": "analyst-9000", "message": "✅ Loaded settings", "file": "/Users/sublime/Desktop/Giorgos/Personal Projects/langraph-ecommerce-agent/src/analyst_9000/backend/core/config.py", "line": 44, "function": "get_settings", "rid": "-", "taskName": "Task-24"}
{"timestamp": "2025-12-01 19:41:08,447", "level": "INFO", "logger": "analyst-9000", "message": "✅ Setup completed successfully", "file": "/Users/sublime/Desktop/Giorgos/Personal Projects/langraph-ecommerce-agent/src/analyst_9000/backend/core/setup.py", "line": 54, "function": "setup", "rid": "-", "taskName": "Task-24"}


'{"orders": "Table `orders`:\\nDescription: Programatically generated orders for The Look fictitious e-commerce store\\n\\n  - order_id (INT64, NULLABLE)\\n  - user_id (INT64, NULLABLE)\\n  - status (STRING, NULLABLE)\\n  - gender (STRING, NULLABLE)\\n  - created_at (TIMESTAMP, NULLABLE)\\n  - returned_at (TIMESTAMP, NULLABLE)\\n  - shipped_at (TIMESTAMP, NULLABLE)\\n  - delivered_at (TIMESTAMP, NULLABLE)\\n  - num_of_item (INT64, NULLABLE)", "order_items": "Table `order_items`:\\nDescription: Programatically generated order items for The Look fictitious e-commerce store\\n\\n  - id (INT64, NULLABLE)\\n  - order_id (INT64, NULLABLE)\\n  - user_id (INT64, NULLABLE)\\n  - product_id (INT64, NULLABLE)\\n  - inventory_item_id (INT64, NULLABLE)\\n  - status (STRING, NULLABLE)\\n  - created_at (TIMESTAMP, NULLABLE)\\n  - shipped_at (TIMESTAMP, NULLABLE)\\n  - delivered_at (TIMESTAMP, NULLABLE)\\n  - returned_at (TIMESTAMP, NULLABLE)\\n  - sale_price (FLOAT64, NULLABLE)", "products": "Table `

In [ ]:
import pprint 

pprint.pprint(settings.bigquery_handler.tables_description)

In [ ]:
print(settings.bigquery_handler.tables_description)

In [ ]:
print(settings.bigquery_handler.tables_description)

In [ ]:
import json
json.dumps(bigquery_handler.tables_description)